In [ ]:
import matplotlib.pyplot as plt
import numpy as np
! pip install tqdm
import tqdm
import pandas as pd
import glob

In [ ]:
def SEA(flares_file_name, data_path, window=60, flare_classes=['M','X'], min_longitude=60):
    '''
    Performs superposed epoch analysis (also known as Chree analysis) on all available NMDB data, with epochs defined from GOES
    EPS data.
    
    Arguments
    flares_file_name:
    The file path of the file containing processed GOES X-ray flare data. This file should be generated by the read_EPS_data
    function in GOES_access_data.ipynb.
    data_path:
    The path of the directory containing files from which to draw neutron count rate data for SEA. These files should have been
    generated by the download_EPS_flare_data function in NMDB_access_data.ipynb, and should all be stored in the same directory
    as each other.
    window:
    The period in minutes after the flare to consider for SEA. This has no effect on the shape of the SEA output curve, but
    should be wide enough to see all relevant information and narrow enough to see detail.
    flare_classes:
    List-like whose elements are strings corresponding to X-ray flare classes. These should be
    the classes you want to examine. For instance, if you only want to examine C-class flares, the only element of 
    this object should be 'C'. More energetic flares are more likely to be associated with neutron production. By
    default, this is equal to ['M','X']. This does not support subclasses, e.g. valid_flare_classes=['M5'] is not
    allowed.
    min_longitude: The minimum absolute value of longitude above which to consider events. X-ray events on the solar
    limb (closer to min_longitude=90) may be more likely to be associated with neutron signals in NMDB. Can be of
    type float or int. By default, this is 60.
    
    Returns
    X_sum: A pandas series object containing the superposition of all valid time series. It has length equal to the size of the
    window arg.
    '''
    
    flare_df = pd.read_csv('GOES_SEP_flares.txt',index_col=0) #Read in the flare information as a dataframe
    file_list = glob.glob(data_path+'/*') #Create a list of the files from which to draw series as inputs to the SEA
    
    X = pd.DataFrame([]) #Initialize a dataframe
    
    counter = 0 #Used to count timeseries and ignore some. Only the 71st and later series have good (1 minute) resolution
    
    for flare_num in tqdm.trange(len(flare_df.columns)): #Cycle through the list of flares
        
        #Record this flare's day, class, and time of peak SEP flux
        day = flare_df[str(flare_num)]['day'] 
        thisClass = flare_df[str(flare_num)]['class'][0]
        peak_time_utc = flare_df[str(flare_num)]['peak_time']
        
        #Ensures that peak_time_utc and the flare longitude can be made into ints
        try: 
            int(peak_time_utc)
            int(flare_df[str(flare_num)]['location'][-2:])
            ints = True
        except (ValueError, TypeError):
            ints = False

        if ints: #Ignores columns with invalid 'peak_time' and 'location' fields
            peak_time = int(day)*24*60 + int(peak_time_utc[:2])*60 + int(peak_time_utc[2:]) #Converts peak_time_utc into minutes
            if peak_time+window <= 44639: #Prevents taking a range incompatible with flare time for that month
                loc = int(flare_df[str(flare_num)]['location'][-2:]) #Record the flare's absolute value longitude
                if thisClass in flare_classes and loc >= min_longitude: #Only includes flares selected by kwargs
                    #Record month and year of this flare
                    month = flare_df[str(flare_num)]['month']
                    year = flare_df[str(flare_num)]['year']
                    file_name = month + '_' + year + '_NMDB.txt'
                    if data_path+file_name in file_list:
                        range_to_take = np.arange(peak_time,peak_time+window) 
                        this_month = pd.read_csv(data_path+file_name, index_col=0) #Read in the file that contains neutron information for this flare
                        this_month = this_month.take(range_to_take) #Only consider the SEA window of the neutron data

                        column_list = [] #Initialize a list to keep track of column names. This ensures no two columns in the final dataframe share the same name
                        for station in this_month:
                            counter += 1
                            if counter > 70: #Ignores time series with <1 minute resolution
                                column_list.append(day+'_'+month+'_'+year+'_'+station)

                                X = pd.concat([X,pd.DataFrame(this_month[station].values,columns=[counter])], axis=1) #Add this time series of neutron data to the dataframe
                                
    X_sum = X.sum(axis=1) #Superpose all the time series in the dataframe
    return(X_sum)

In [ ]:
sea = SEA(flares_file_name='GOES_SEP_flares.txt', data_path='NMDB_SEP_data')

plt.figure(figsize=(10,4))
plt.plot(np.arange(len(sea)),sea)

plt.title('Superposition of neutron curves following SEP-producing X-ray flares')
plt.xlabel('Time (minutes since flare)')
plt.ylabel('Superposition (summed counts/s)')